In [17]:
import ta
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
import pandas as pd
import plotly.express as px

In [4]:
data = pdr.get_data_yahoo("AAPL", start="2000-01-01", end="2023-01-01")

[*********************100%%**********************]  1 of 1 completed


In [6]:
data.to_csv('AAPL.csv')

In [121]:
data = pd.read_csv('AAPL.csv')
raw_data = data.copy()

In [122]:
px.line(data, x='Date', y='Close')

In [123]:
import pandas as pd
import numpy as np

def apply_npmm_labeling(df, column_name='Close', window_size=10):
    """
    Apply NPMM (N-Period Max-Min) labeling to a pandas DataFrame.

    Parameters:
    - df: pandas DataFrame containing the time series data.
    - column_name: Name of the column containing the time series data to be labeled. Default is 'Close'.
    - window_size: The size of the rolling window for labeling. Default is 10.

    Returns:
    - Labeled DataFrame.
    """

    # Initialize a new column for the label
    df['Label'] = None

    # Loop through the DataFrame to find local maxima and minima
    for i in range(0, len(df) - window_size + 1):
        window = df.loc[i:i+window_size-1, column_name]

        # Find local maximum and minimum in the window
        local_max = window.max()
        local_min = window.min()

        # Check where the local maximum and minimum occur in the window
        max_index = window[window == local_max].index[0]
        min_index = window[window == local_min].index[0]

        # Label the points
        df.loc[max_index, 'Label'] = 0  # 0 for maximum points
        df.loc[min_index, 'Label'] = 1  # 1 for minimum points

    return df

# Apply NPMM labeling
labeled_df = apply_npmm_labeling(data, 'Close', 14)

In [124]:
fig = px.scatter(labeled_df, x='Date', y='Close', color='Label')
# add a line plot to the scatter plot of the Close price
fig.add_trace(px.line(labeled_df, x='Date', y='Close').data[0])

In [125]:
def create_signals(labeled_df):
    prev = None
    prevind = None
    signals = [None] * len(labeled_df['Label'])
    for (i, val) in enumerate(labeled_df['Label']):
        if prev is None and val is not None:
            prev = val
            prevind = i
        if val is None:
            continue
        if prev == 1 and val == 0:
            signals[prevind] = 'Buy'
        if prev == 0 and val == 1:
            signals[prevind] = 'Sell'
        if val is not None:
            prev = val
            prevind = i
    return signals

In [126]:
labeled_df = apply_npmm_labeling(data, 'Close', 14)
labeled_df['Signal'] = create_signals(labeled_df)

In [127]:
fig = px.scatter(labeled_df, x='Date', y='Close', color='Signal')
# add a line plot to the scatter plot of the Close price
fig.add_trace(px.line(labeled_df, x='Date', y='Close').data[0])

In [128]:
full_holds = np.asarray(labeled_df.index[labeled_df['Signal'].isnull()].tolist())
full_buys = np.asarray(labeled_df.index[labeled_df['Signal'] == 'Buy'].tolist())
full_sells = np.asarray(labeled_df.index[labeled_df['Signal'] == 'Sell'].tolist())

In [215]:
ws = 14

In [216]:
holds = full_holds[full_holds >= ws]
buys = full_buys[full_buys >= ws]
sells = full_sells[full_sells >= ws]
holds = np.random.choice(holds, 571, replace=False)

In [217]:
from sklearn.preprocessing import StandardScaler

In [218]:
def gen_varima_col(ind, df, ws):
    return StandardScaler().fit_transform(df.iloc[ind - ws:ind].values).reshape(1, -1)

In [219]:
raw_data = raw_data.set_index('Date')

KeyError: "None of ['Date'] are in the columns"

In [220]:
buys_data = np.vstack([gen_varima_col(i, raw_data, ws) for i in buys])
sells_data = np.vstack([gen_varima_col(i, raw_data, ws) for i in sells])
holds_data = np.vstack([gen_varima_col(i, raw_data, ws) for i in holds])
full_data = np.vstack([buys_data, sells_data, holds_data])

In [221]:
target = [0] * buys_data.shape[0] + [1] * sells_data.shape[0] + [2] * holds_data.shape[0]
target = np.asarray(target)

In [222]:
from sklearn.model_selection import train_test_split

In [223]:
train_x, test_x, train_y, test_y = train_test_split(full_data, target, test_size=0.2, random_state=42)

In [236]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

In [240]:
clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(train_x, train_y)
clf.score(test_x, test_y), clf.score(train_x, train_y)

(0.35276967930029157, 0.6922514619883041)

In [242]:
just_buy_sell = np.vstack([buys_data, sells_data])
just_buy_sell_target = [0] * buys_data.shape[0] + [1] * sells_data.shape[0]
just_buy_sell_target = np.asarray(just_buy_sell_target)

In [243]:
train_x, test_x, train_y, test_y = train_test_split(just_buy_sell, just_buy_sell_target, test_size=0.2, random_state=42)

In [297]:
clf = XGBClassifier(max_depth = 3, n_estimators = 20).fit(train_x, train_y)
probs = clf.predict_proba(test_x)
pred = clf.predict(test_x)
clf.score(test_x, test_y), clf.score(train_x, train_y)

(0.5833333333333334, 0.8739035087719298)

In [270]:
def var(ps):
    return ps[0,:]*ps[1]

In [291]:
np.mean(test_y[(probs[:,0] > .8) | (probs[:,1] > .8)] != pred[(probs[:,0] > .8) | (probs[:,1] > .8)])

0.37037037037037035